### IMPORT LIBRARIES & SETTINGS

In [1]:
# Install Required Packages
# ! pip -q install nselib
# ! pip -q install yfinance

In [2]:
# Import Libraries
import os
import warnings
import yfinance
import numpy as np
import datetime as dt
import pandas as pd
from nselib import capital_market
warnings.filterwarnings('ignore')
pd.set_option('mode.chained_assignment', None)

In [3]:
# Set Current Directory
os.chdir('C://Users//Riddhik//OneDrive//Desktop//Personal//Financial Planning//Data//')
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: C:\Users\Riddhik\OneDrive\Desktop\Personal\Financial Planning\Data


In [4]:
# Get Today's Date
dt_today = str(dt.datetime.today())
curr_year = dt_today[:4]
curr_month = dt_today[5:7]
curr_date = dt_today[8:10]
save_date =  curr_year + "_" + curr_month + "_" + curr_date
print("Today's Date: " + curr_year + "/" + curr_month + "/" + curr_date)

Today's Date: 2023/07/05


In [5]:
# Date Manipulation
START_YEAR = 1997; START_MONTH = 1; START_DAY = 1
END_YEAR = int(curr_year); END_MONTH = int(curr_month); END_DAY = int(curr_date)
final_start_date = dt.date(START_YEAR, START_MONTH, START_DAY)
final_end_date = dt.date(END_YEAR, END_MONTH, END_DAY)
print("Data pull start date:", final_start_date)
print("Data pull end date: ", final_end_date)

Data pull start date: 1997-01-01
Data pull end date:  2023-07-05


### READ INPUT FILE

In [6]:
# Read Index Input File
input_df = pd.read_csv('./input_file.csv')
nse_py_data = input_df[input_df['EXTRACT_DATA'] == 1]
y_fin_data = input_df[input_df['EXTRACT_DATA'] == 2]
del input_df

In [7]:
# Show Counts
print("Count of Files nse_py: ", len(nse_py_data['INDEX'].to_list()))
print("Count of Files yfin: ", len(y_fin_data['INDEX'].to_list()))

Count of Files nse_py:  38
Count of Files yfin:  6


### HELPER FUNCTIONS

In [8]:
def create_skeleton(final_start_date, final_end_date):
    '''Create Weekly Skeleton'''
    
    skeleton = pd.DataFrame(pd.date_range(final_start_date, final_end_date, freq='W-FRI'))
    skeleton.columns = ['Date']
    
    return skeleton

In [9]:
def rec_impute(skeleton_df, csv_dataset, index_close_df, day):
    '''Create proxy dates for missing values for imputing Close values'''
    
    null_df_index = csv_dataset[csv_dataset['Close'].isnull()]
    null_df_skeleton = skeleton_df[skeleton_df['Date'].isin(null_df_index.Date)]
    null_df_skeleton['Proxy_Date'] = null_df_skeleton['Date'] - dt.timedelta(days = day)
    imputed_df = pd.merge(null_df_skeleton, index_close_df, left_on = 'Proxy_Date',
                          right_on = 'Date', how = 'left')
    imputed_df.drop(['Proxy_Date', 'Date_y'], axis = 1, inplace = True)  
    imputed_df.columns = ['Date', 'Close']
    
    return imputed_df

In [10]:
def impute_data(skeleton_df, csv_dataset, index_close_df):
    '''Impute for 4 days from Friday if Close value not found'''
    
    # create impute data upto 4 previous close values
    impute_1 = rec_impute(skeleton_df, csv_dataset, index_close_df, 1)
    impute_2 = rec_impute(skeleton_df, impute_1, index_close_df, 2)
    impute_3 = rec_impute(skeleton_df, impute_2, index_close_df, 3)
    impute_4 = rec_impute(skeleton_df, impute_3, index_close_df, 4)
    impute_1.dropna(inplace = True); impute_2.dropna(inplace = True)
    impute_3.dropna(inplace = True); impute_4.dropna(inplace = True)
    final_impute_df = pd.concat([impute_1, impute_2, impute_3, impute_4])
    
    # create final inpute dataframe
    fin_df = pd.merge(csv_dataset, final_impute_df, on = 'Date', how = 'left')
    fin_df.Close_x.fillna(fin_df.Close_y, inplace=True)
    fin_df.drop(['Close_y'], axis = 1, inplace = True)
    fin_df.columns = ['Date', 'Close']
    
    return fin_df

In [11]:
def get_full_history_nse_py(df, index_name):
    '''Return DAILY historical close data of the index from nse'''
    
    for year in range(1994, int(curr_year)+1):
        
        from_date_str = '01-01-' + str(year)
        to_date_str = str(curr_date) + "-" + str(curr_month)  + "-" + str(curr_year)
        
        try:
            df = capital_market.index_data(index=index_name, from_date=from_date_str, to_date=to_date_str)
            df = df[['TIMESTAMP', 'CLOSE_INDEX_VAL', 'INDEX_NAME']]
            break 
        except:    
            pass

    df.rename(columns={'TIMESTAMP':'Date', 'CLOSE_INDEX_VAL':'Close', 'INDEX_NAME':'Index'}, inplace=True)
    df = df.round({'Close':0})
    df.drop_duplicates(subset = ['Date'], keep = 'first', inplace = True)
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
    df.drop_duplicates(subset = ['Date'], keep = 'first', inplace = True)
    
    return df    

In [12]:
# CHECK ETF/TICKER
#get_full_history_nse_py(nse_py_data, 'Nifty50 Value 20')

In [13]:
def get_full_history_yfin(etf_name, start_date, end_date):
    '''Return DAILY historical close data of the index from yahoo finance'''
    
    ticker = yfinance.Ticker(etf_name)
    data = ticker.history(interval = '1d', start = start_date,
                          end = end_date, actions = False)
    data = data.reset_index()
    data = data[['Date', 'Close']]
    data = data[data['Close'].notna()]
    data['Date']= pd.to_datetime(data['Date'])
    data = data.round({'Close':0})
    data.drop_duplicates(subset = ['Date'], keep = 'first', inplace = True)
    
    return data

In [14]:
# CHECK ETF/TICKER
#get_full_history_yfin('SILVERBEES.NS', '2023-01-01', '2023-02-01')

In [15]:
def create_weekly_data(dataset, from_start_date, from_end_date):
    '''Create Weekly data after pulling daily data'''
    
    # Create skeleton
    skeleton_df = create_skeleton(from_start_date, from_end_date)
    
    # merge with skeleton
    csv_dataset = pd.merge(skeleton_df, dataset, on = 'Date', how = 'left')

    # filter non-existent data
    csv_dataset = csv_dataset[csv_dataset.Date >= min(dataset.Date)]
    csv_dataset = impute_data(skeleton_df, csv_dataset, dataset)

    # forward fill missing values that could not be imputed
    csv_dataset.fillna(method='ffill', inplace=True)
    csv_dataset = csv_dataset[csv_dataset['Close'].notna()]
    
    return csv_dataset

### EXTRACT DATA

In [16]:
# Yahoo Finance
for etf_name in y_fin_data['ETF'].tolist():
    
    # Get Daily Data
    daily_data = get_full_history_yfin(etf_name, start_date=final_start_date, end_date=final_end_date)
    daily_data['Date'] = pd.to_datetime(daily_data['Date']).dt.tz_localize(None)
    min_date = str(daily_data['Date'].min())[:10]
    max_date = str(daily_data['Date'].max())[:10]
    daily_data.to_csv('./daily/' + etf_name + "_" + min_date + "_" + max_date + ".csv", index=False)
    del min_date, max_date
    print("Saved daily data: " + etf_name)
    
    # Create Weekly Data
    data = create_weekly_data(daily_data, final_start_date, final_end_date)
    min_date = str(data['Date'].min())[:10]
    max_date = str(data['Date'].max())[:10]
    data['Index'] = etf_name
    
    # data cleanup
    if etf_name == 'GOLDBEES.NS':
        data.loc[data['Date'] == '2019-12-20', 'Close'] = 33.65
    if etf_name == 'HNGSNGBEES.NS':
        data.loc[data['Date'] == '2019-12-20', 'Close'] = 368.94
    if etf_name == 'ICICI500.NS':
        data_p1 = data[data['Date'] <= '2021-10-22']
        data_p2 = data[data['Date'] >= '2021-10-29']
        data_p1['Close'] = data_p1['Close'].div(10)
        data = pd.concat([data_p1, data_p2])
        data.loc[data['Date'] == '2021-10-29', 'Close'] = 24.99
        del data_p2, data_p1
    if etf_name == 'ICICIMCAP.NS':
        data = data[data['Date'] >= '2019-02-15']
    
    data.to_csv('./weekly/' + etf_name + "_" + min_date + "_" + max_date + ".csv", index=False)
    del min_date, max_date
    print("Saved weekly data: " + etf_name)
    print()

del daily_data, data

Saved daily data: ICICI500.NS
Saved weekly data: ICICI500.NS

Saved daily data: ICICIMCAP.NS
Saved weekly data: ICICIMCAP.NS

Saved daily data: GOLDBEES.NS
Saved weekly data: GOLDBEES.NS

Saved daily data: ICICIB22.NS
Saved weekly data: ICICIB22.NS

Saved daily data: SILVERBEES.NS
Saved weekly data: SILVERBEES.NS

Saved daily data: SHARIABEES.NS
Saved weekly data: SHARIABEES.NS



In [17]:
# NSE Data
for index_name in nse_py_data['INDEX'].tolist():
    
    # Get Daily Data
    daily_data = get_full_history_nse_py(nse_py_data, index_name)
    min_date = str(daily_data['Date'].min())[:10]
    max_date = str(daily_data['Date'].max())[:10]
    daily_data.to_csv('./daily/' + index_name + "_" + min_date + "_" + max_date + ".csv", index=False)
    daily_data.drop(columns=['Index'], inplace=True)
    print("Saved daily data: " + index_name)
    del min_date, max_date
    
    # Create Weekly Data
    data = create_weekly_data(daily_data, final_start_date, final_end_date)
    min_date = str(data['Date'].min())[:10]
    max_date = str(data['Date'].max())[:10]
    data['Index'] = index_name
    data.to_csv('./weekly/' + index_name +  "_" + min_date + "_" + max_date + ".csv", index=False)
    del min_date, max_date
    print("Saved weekly data: " + index_name)
    print()

del daily_data, data

Saved daily data: NIFTY 100
Saved weekly data: NIFTY 100

Saved daily data: NIFTY 50
Saved weekly data: NIFTY 50

Saved daily data: NIFTY LARGEMIDCAP 250
Saved weekly data: NIFTY LARGEMIDCAP 250

Saved daily data: NIFTY MICROCAP 250
Saved weekly data: NIFTY MICROCAP 250

Saved daily data: NIFTY MIDCAP 100
Saved weekly data: NIFTY MIDCAP 100

Saved daily data: NIFTY MIDCAP 150
Saved weekly data: NIFTY MIDCAP 150

Saved daily data: NIFTY MIDCAP 50
Saved weekly data: NIFTY MIDCAP 50

Saved daily data: NIFTY NEXT 50
Saved weekly data: NIFTY NEXT 50

Saved daily data: NIFTY SMALLCAP 250
Saved weekly data: NIFTY SMALLCAP 250

Saved daily data: NIFTY SMALLCAP 50
Saved weekly data: NIFTY SMALLCAP 50

Saved daily data: NIFTY AUTO
Saved weekly data: NIFTY AUTO

Saved daily data: NIFTY BANK
Saved weekly data: NIFTY BANK

Saved daily data: NIFTY FINANCIAL SERVICES
Saved weekly data: NIFTY FINANCIAL SERVICES

Saved daily data: NIFTY FMCG
Saved weekly data: NIFTY FMCG

Saved daily data: NIFTY HEALTH